In [1]:
import pandas as pd
from community import community_louvain

merged = pd.read_csv('ubs-lauzhack-2024/external_parties_train.csv')
merged.head()

/opt/homebrew/anaconda3/envs/adaexam/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


,transaction_reference_id,party_role,party_info_unstructured,parsed_name,parsed_address_street_name,parsed_address_street_number,parsed_address_unit,parsed_address_postal_code,parsed_address_city,parsed_address_state,parsed_address_country,party_iban,party_phone,external_id
0,04ff0d1c680189e3a80c92d86407f0f5,BENE,mary mith 107 107 angela brooks n. thomasfurt ...,mary mith,angela brooks,107 107,NaN,NaN,n. thomasfurt,NaN,bulgaria,GB49MYOB82127728573340,+1.815660-6791x8486,50039037
1,439ab0ad7380e6135ab2ff3fddd4a727,ORG,yesneia kim north michael 93971 koribati,yesneia kim,north michael,NaN,NaN,93971,koribati,NaN,NaN,NaN,0 (269)620-8734x2349,60044692
2,00cac12d41191a84f9e31aa731a83512,ORG,w. roberson jr. 41010 rachel crossingapt. 923 ...,w. roberson jr.,rachel crossingapt.,41010 923,NaN,p2235417,thompsonshire amyport,NaN,NaN,GB08OTHR53515837682953,NaN,30008244
3,e4fba5f878dd3453e35973605a783a16,BENE,azquez-nelson co. suarez ports suite & 024 bri...,azquez-nelson co.,ports suite &,NaN,NaN,NaN,brittanyberg,NaN,bulgaria bulgaria,GB17VVGW66321494633280,NaN,40017944
4,d03d7e4c31878b0255d39e8c3f0ab625,ORG,m.j. bytd iii 856 john lake s. glenn cocos (ke...,m.j. bytd iii,john lake s. glenn,856,NaN,125838276,cocos (keeling),islands,NaN,NaN,(260)3371534,40012658


In [2]:
import re

def clean_phone_numbers(phone_numbers):
    """
    Cleans a list of phone numbers by removing 
    alphabetical characters, dashes, underscores, spaces, and plus signs, ...
    
    Args:
        phone_numbers (list of str): List of phone numbers to clean.
    
    Returns:
        list of str: Cleaned phone numbers.
    """
    cleaned_numbers = [re.sub(r'[a-zA-Z\-\_\s\+\.\,\'\^\(\)]', '', str(phone)) for phone in phone_numbers]
    return cleaned_numbers

In [3]:
# Get from geopandas the list of all countries
import geopandas as gpd
from rapidfuzz import process, fuzz

# Compare each country with the list of countries in world and compute the similarity
def get_closest_country(name):
    country = process.extract(name, world['name'], scorer=fuzz.ratio, limit=3)
    for c in country:
        if country[0][1] > 85:
            return c[0]
    return None

# Get the list of the countries in the world
world = gpd.read_file("https://naciscdn.org/naturalearth/110m/cultural/ne_110m_admin_0_countries.zip").rename(columns={'ADMIN': 'name'})
world['name'] = world['name'].str.lower()


# For the countries where the length is even check if the same name is repeated twice. If so fix it
countries = merged['parsed_address_country'].dropna().unique()
for country in countries:
    if  len(country.split(" ")) % 2 == 0:
        country_split = country.split(" ")
        if ' '.join(country_split[:len(country_split)//2]) == ' '.join(country_split[len(country_split)//2:]):
            merged['parsed_address_country'] = merged['parsed_address_country'].replace(country, ' '.join(country_split[:len(country_split)//2]))

# Get the list of countries that are not in the world list
countries = merged['parsed_address_country'].dropna().unique()
remapping = {}
to_remove = []
for country in countries:
    results = get_closest_country(country)
    if results is not None:
        remapping[country] = results
    else:
        to_remove.append(country)


In [4]:
from collections import defaultdict
import re

def normalize_name(name):
    # Convert to lowercase and strip whitespace
    name = name.lower().strip()
    
    # Define all patterns to remove
    patterns = {
        # Titles
        r'\b(dr\.|mr\.|mrs\.|ms\.|prof\.)\s*': '',
        # Suffixes 
        r'\s+(jr\.|sr\.|ii|iii|md|dvm)\b': '',
        # Business entities
        r'\s+(ltd|llc|inc|corp|s\.a\.r\.l\.|p\.l\.c\.|group|grp\.)\b': '',
        # Multiple spaces
        r'\s+': ' '
    }
    
    # Apply all patterns in a single pass
    for pattern, repl in patterns.items():
        name = re.sub(pattern, repl, name)
        
    return name



In [5]:
    
# Remap the countries
merged['parsed_address_country'] = merged['parsed_address_country'].replace(remapping)

print('replaced countries')

# Put to nan the countries that are in the to_remove list
merged['parsed_address_country'] = merged['parsed_address_country'].replace(to_remove, None)

merged['party_phone'] = clean_phone_numbers(merged['party_phone'])

print('cleaned phone numbers')

merged['parsed_name'] = merged['parsed_name'].apply(normalize_name)



replaced countries
cleaned phone numbers


In [6]:
merged.to_csv('train_cleaned.csv', index=False)

In [6]:
# merged.to_csv('test_cleaned.csv', index=False)

In [37]:
import pandas as pd
# merged = pd.read_csv('test_cleaned.csv')

filtered = merged[['transaction_reference_id', 'parsed_name', 'parsed_address_country', 'party_phone', 'party_iban', 'external_id']]
filtered

,transaction_reference_id,parsed_name,parsed_address_country,party_phone,party_iban,external_id
0,04ff0d1c680189e3a80c92d86407f0f5,mary mith,bulgaria,181566067918486,GB49MYOB82127728573340,50039037
1,439ab0ad7380e6135ab2ff3fddd4a727,yesneia kim,NaN,026962087342349,NaN,60044692
2,00cac12d41191a84f9e31aa731a83512,w. roberson jr.,NaN,,GB08OTHR53515837682953,30008244
3,e4fba5f878dd3453e35973605a783a16,azquez-nelson co.,bulgaria,,GB17VVGW66321494633280,40017944
4,d03d7e4c31878b0255d39e8c3f0ab625,m.j. bytd,NaN,2603371534,NaN,40012658
...,...,...,...,...,...,...
11059,7183d9c3700148c9527869948b685085,james alvarado jr.,zambia,4194922033879822,GB72FLST90715739139871,35031814
11060,b47b9ed0a8cc9fcc4c21ac368fe79757,marcnguyen,NaN,,GB64SFWQ57121822291812,35003780
11061,d61ff2b0184f32ad0021a313c6112b2e,joseph davis,NaN,004118784377081,GB36XGTI64012968550973,30014262
11062,bec335b1b1bad8c55b7dce549cfd8de0,gonzalezltd,NaN,,NaN,10359417


In [29]:
import pandas as pd
import networkx as nx
from collections import defaultdict
from tqdm import tqdm

def create_weighted_similarity_graph(df, column_weights):
    # Initialize graph
    G = nx.Graph()
    
    # Create mapping from index to transaction ID
    idx_to_txn = dict(zip(df.index, df['transaction_reference_id']))
    
    # Add all rows as nodes first, using transaction IDs
    for idx in tqdm(df.index, desc="Adding nodes"):
        G.add_node(idx_to_txn[idx])
    
    # Create value-to-nodes mapping for each column
    column_mappings = {}
    for column in tqdm(column_weights.keys(), desc="Processing columns"):
        if column != 'transaction_reference_id':
            column_mappings[column] = defaultdict(list)
            
            # For each value in the column, store the transaction IDs that contain it
            for idx, value in tqdm(df[column].items(), desc=f"Processing {column}", leave=False):
                if pd.notna(value) and value != '':  # Skip empty/NaN values
                    column_mappings[column][value].append(idx_to_txn[idx])
    
    # Create edges between nodes that share values, with weights
    for column, value_map in tqdm(column_mappings.items(), desc="Creating edges"):
        weight = column_weights[column]  # Get weight from column_weights
        for nodes in tqdm(value_map.values(), desc=f"Processing {column} values", leave=False):
            # If multiple nodes share this value, create edges between them
            if len(nodes) > 1:
                for i in range(len(nodes)):
                    for j in range(i + 1, len(nodes)):
                        G.add_edge(nodes[i], nodes[j], weight=weight)
    
    return G

# Example usage
column_weights = {
    'parsed_name': 0.2,
    # 'parsed_address_country': 0,
    'party_phone': 1,
    'party_iban': 1,
}


no_label = filtered.drop(columns=['external_id']) if 'external_id' in filtered.columns else filtered


# Create the graph with weights
G = create_weighted_similarity_graph(no_label, column_weights)

# If you want to convert back to a DataFrame showing connections with weights:
edges_df = pd.DataFrame([(u, v, d['weight']) for u, v, d in tqdm(G.edges(data=True), desc="Creating DataFrame")], columns=['node1', 'node2', 'weight'])

Processing columns:   0%|          | 0/3 [00:00<?, ?it/s]
Processing parsed_name: 0it [00:00, ?it/s]

Creating DataFrame: 100%|██████████| 11756/11756 [00:00<00:00, 1501971.97it/s]


In [27]:
edges_df

,node1,node2,weight
0,dc9be336b81b971c04ca98ccdf99d51e,4a21d5917c868aac428ee1ad3ba1bfd5,0.2
1,dc9be336b81b971c04ca98ccdf99d51e,7312d4c0ffa799ddbb245f3af214b784,0.2
2,dc9be336b81b971c04ca98ccdf99d51e,caff907068d4e79dfa7c75c15ab247a5,0.2
3,dc9be336b81b971c04ca98ccdf99d51e,e3a5263d7dcb1edf60b370cfa54718ef,0.2
4,29c8ed895b8a220121168158a7447773,d0586422551c55de3cd50c71446889bd,0.2
...,...,...,...
31546863,c33ae0d30b7900804318ec731b42ef12,288567ebe983af45fed859775c59dd2f,0.2
31546864,ed9dc263f0a535ed9a3b28b5a2041f7c,ef05c2967f5d0ac3a411f5a3b9ba8c4a,0.2
31546865,5ffc7d74f9b31dbbd84ffee4bad120d7,f8fa5a3bab3e73a64af4d1ddb51fb3e3,0.2
31546866,831e5790ae09bfc99b7a258f4f324e15,9c2d7ee48e78c77089edaf14694826d8,0.2


In [30]:

import matplotlib.pyplot as plt
from networkx import connected_components

# Get the connected components
components = list(connected_components(G))

# Create a dictionary mapping transaction IDs to component numbers
component_mapping = {}
for i, component in enumerate(components):
    for node in component:
        component_mapping[node] = i

# Convert to DataFrame
component_df = pd.DataFrame.from_dict(component_mapping, orient='index', columns=['component_id'])
component_df.index.name = 'transaction_reference_id'
component_df = component_df.reset_index()

print(f"Number of connected components: {len(components)}")
print(f"Size of largest component: {len(max(components, key=len))}")
print(f"Size of smallest component: {len(min(components, key=len))}")

# Display first few rows of the mapping
print("\nFirst few transaction ID to component mappings:")
print(component_df.head())


Number of connected components: 5799
Size of largest component: 22
Size of smallest component: 1

First few transaction ID to component mappings:
           transaction_reference_id  component_id
0  04ff0d1c680189e3a80c92d86407f0f5             0
1  439ab0ad7380e6135ab2ff3fddd4a727             1
2  522adb14791b271b6947c79258c9c795             2
3  00cac12d41191a84f9e31aa731a83512             2
4  cf49e7b1d82830689ca52246aa1f174d             2


In [31]:
# Get sizes of 20 largest components
largest_components = sorted(components, key=len, reverse=True)[:20]
component_sizes = [len(comp) for comp in largest_components]

print("Sizes of 20 largest components:")
for i, size in enumerate(component_sizes, 1):
    print(f"Component {i}: {size} nodes")


Sizes of 20 largest components:
Component 1: 22 nodes
Component 2: 17 nodes
Component 3: 15 nodes
Component 4: 14 nodes
Component 5: 13 nodes
Component 6: 12 nodes
Component 7: 12 nodes
Component 8: 12 nodes
Component 9: 11 nodes
Component 10: 11 nodes
Component 11: 11 nodes
Component 12: 10 nodes
Component 13: 10 nodes
Component 14: 10 nodes
Component 15: 10 nodes
Component 16: 10 nodes
Component 17: 10 nodes
Component 18: 10 nodes
Component 19: 9 nodes
Component 20: 8 nodes


In [32]:
# Create a dictionary to store the final cluster assignments
final_clusters = {}

# Get the original data with IBANs
iban_data = filtered[['transaction_reference_id', 'party_iban']].dropna()
iban_dict = dict(zip(iban_data['transaction_reference_id'], iban_data['party_iban']))

# Sort components by size and get 10 largest
sorted_components = sorted(components, key=len, reverse=True)
top_10_components = sorted_components[:4000]
other_components = sorted_components[4000:]

# Process 10 largest components
for i, component in tqdm(enumerate(top_10_components), desc="Processing components"):
    component_nodes = list(component)
    
    # Group nodes by IBAN
    iban_groups = {}
    for node in component_nodes:
        iban = iban_dict.get(node)
        if iban:
            if iban not in iban_groups:
                iban_groups[iban] = []
            iban_groups[iban].append(node)
    
    # Assign cluster IDs
    max_cluster = max(final_clusters.values()) if final_clusters else -1
    for iban, nodes in iban_groups.items():
        cluster_id = max_cluster + 1
        for node in nodes:
            final_clusters[node] = cluster_id
        max_cluster = cluster_id
        
    # Nodes without IBAN get individual clusters
    for node in component_nodes:
        if node not in final_clusters:
            max_cluster += 1
            final_clusters[node] = max_cluster

# Process remaining components - assign one cluster per component
max_cluster = max(final_clusters.values()) if final_clusters else -1
for component in other_components:
    max_cluster += 1
    for node in component:
        final_clusters[node] = max_cluster

# Convert to DataFrame
clusters_df = pd.DataFrame.from_dict(final_clusters, orient='index', columns=['external_id'])
clusters_df.index.name = 'transaction_reference_id'
clusters_df = clusters_df.reset_index()

print("Number of unique clusters:", len(set(final_clusters.values())))
clusters_df.head()


Processing components: 4000it [00:00, 10939.70it/s]

Number of unique clusters: 8330


,transaction_reference_id,external_id
0,37fad659e3bdb05f9ccc456abb9c6b95,0
1,e4bd2988d2f1798e806fa432aa0fa3cd,1
2,9705f8d5209f0c261dead8cca87f3f2d,1
3,8f6caea0a018a92ee98e77c79f3da0a1,1
4,9a6f981146f68100ed4ea807ab06041f,1


In [33]:
clusters_df.external_id.value_counts()

external_id
300     6
248     6
885     6
487     6
877     6
       ..
3415    1
3414    1
3413    1
3412    1
8329    1
Name: count, Length: 8330, dtype: int64

In [22]:
# clusters_df.to_csv('clusters_df.csv', index=False)

In [14]:

def louvain_clustering(G):
    # Get all nodes from the original dataset
    all_nodes = set(filtered['transaction_reference_id'])
    
    # Get nodes that are in the graph
    graph_nodes = set(G.nodes())
    
    # Find isolated nodes (not in graph)
    isolated_nodes = all_nodes - graph_nodes
    print("All nodes: ", len(all_nodes))
    print("Graph nodes: ", len(graph_nodes))
    print("Isolated nodes: ", len(isolated_nodes)) 
    
    # Perform Louvain community detection on the graph
    communities = community_louvain.best_partition(G)
    

    # Get the maximum community ID used
    max_community_id = max(communities.values()) if communities else -1
    
    # Assign unique IDs to isolated nodes
    for i, node in enumerate(isolated_nodes):
        communities[node] = max_community_id + 1 + i
    
    # Convert results to DataFrame
    clusters_df = pd.DataFrame.from_dict(communities, orient='index', columns=['cluster'])

    return clusters_df






Largest component size: 22 nodes
Original graph size: 11064 nodes
Nodes not in largest component: 11042


In [19]:
# Run clustering on largest component
clusters_louvain_largest = louvain_clustering_largest_component(G)

clusters_louvain_largest = clusters_louvain_largest.rename(columns={'cluster': 'external_id'})
clusters_louvain_largest = clusters_louvain_largest.reset_index()
clusters_louvain_largest = clusters_louvain_largest.rename(columns={'index': 'transaction_reference_id'})
clusters_louvain_largest

Largest component size: 22 nodes
Original graph size: 11064 nodes
Nodes not in largest component: 11042


,transaction_reference_id,external_id
0,e4bd2988d2f1798e806fa432aa0fa3cd,0
1,03b1304d25e9c39370023b2d1d16263f,1
2,9705f8d5209f0c261dead8cca87f3f2d,0
3,37fad659e3bdb05f9ccc456abb9c6b95,1
4,8a86ef88d7d675f4bb9ecf6792ae0943,2
...,...,...
11059,208a55c2f08f567d36cfa0dbb02152e4,11040
11060,0f999cdf6562ae7e5e458f35dc21afc7,11041
11061,fdb1f14d769c18078abdda1856db5235,11042
11062,ecf7fd59986641f04a8087d5a8d626ba,11043


In [16]:
clusters_louvain = louvain_clustering(G)

clusters_louvain = clusters_louvain.rename(columns={'cluster': 'external_id'})
clusters_louvain = clusters_louvain.reset_index()
clusters_louvain = clusters_louvain.rename(columns={'index': 'transaction_reference_id'})
clusters_louvain

All nodes:  11064
Graph nodes:  11064
Isolated nodes:  0


,transaction_reference_id,external_id
0,04ff0d1c680189e3a80c92d86407f0f5,0
1,439ab0ad7380e6135ab2ff3fddd4a727,1
2,00cac12d41191a84f9e31aa731a83512,2
3,e4fba5f878dd3453e35973605a783a16,3
4,d03d7e4c31878b0255d39e8c3f0ab625,4
...,...,...
11059,7183d9c3700148c9527869948b685085,701
11060,b47b9ed0a8cc9fcc4c21ac368fe79757,720
11061,d61ff2b0184f32ad0021a313c6112b2e,250
11062,bec335b1b1bad8c55b7dce549cfd8de0,5798


In [38]:
def compute_clustering_metrics(df1, df2):
    """
    Compute precision, recall, and F1 score for clustering assignments
    
    Args:
        df1: DataFrame with columns [transaction_reference_id, external_id]
        df2: DataFrame with columns [transaction_reference_id, external_id]
    
    Returns:
        dict: Contains precision, recall, and F1 score
    """
    # Create dictionaries mapping transaction_id to cluster_id for both dataframes
    clusters1 = dict(zip(df1['transaction_reference_id'], df1['external_id']))
    clusters2 = dict(zip(df2['transaction_reference_id'], df2['external_id']))
    
    # Get all unique transaction IDs
    all_ids = list(clusters1.keys())
    n = len(all_ids)
    
    # Initialize counters for confusion matrix
    true_positive = 0
    false_positive = 0
    false_negative = 0
    
    # Compare all possible pairs
    for i in tqdm(range(n)):
        for j in range(i + 1, n):
            id1, id2 = all_ids[i], all_ids[j]
            
            # Check if pairs are in same cluster in both assignments
            same_cluster1 = clusters1[id1] == clusters1[id2]
            same_cluster2 = clusters2[id1] == clusters2[id2]
            
            if same_cluster1 and same_cluster2:
                true_positive += 1
            elif same_cluster1 and not same_cluster2:
                false_positive += 1
            elif not same_cluster1 and same_cluster2:
                false_negative += 1
    
    # Calculate metrics
    precision = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0
    recall = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return {
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

# Usage:
metrics = compute_clustering_metrics(filtered, clusters_df)
print(f"Precision: {metrics['precision']:.3f}")
print(f"Recall: {metrics['recall']:.3f}")
print(f"F1 Score: {metrics['f1']:.3f}")

100%|██████████| 11064/11064 [00:07<00:00, 1442.54it/s]

Precision: 0.381
Recall: 1.000
F1 Score: 0.552
